In [1]:
import json
from kafka import KafkaConsumer
from pymongo import MongoClient
from FraudModel import runModel  # Mengimpor fungsi runModel dari predict_model.py
import warnings
warnings.filterwarnings("ignore")
data = []


In [2]:
import pandas as pd
import urllib.parse

from sqlalchemy import create_engine
# Informasi koneksi ke PostgreSQL
username = "postgres"
password = "123456789"
host = "localhost"
port = "5432"
database = "project-4"
password = urllib.parse.quote_plus(password)

# URL koneksi ke PostgreSQL
db_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"

engine = create_engine(db_url)

In [3]:
df = pd.read_sql_query('SELECT * FROM old_information;', engine)
df

,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest
0,C1231006815,170136.0,M1979787155,0.00
1,C1666544295,21249.0,M2044282225,0.00
2,C1305486145,181.0,C553264065,0.00
3,C840083671,181.0,C38997010,21182.00
4,C2048537720,41554.0,M1230701703,0.00
...,...,...,...,...
14995,C1467327074,29051.0,C191614132,9701.14
14996,C932879787,0.0,M902010316,0.00
14997,C1430251794,0.0,M799225701,0.00
14998,C1287098623,55498.0,M1702039888,0.00


In [6]:
if __name__ == "__main__":
    consumer = KafkaConsumer("ftde01-project4-pipeline_wizard", bootstrap_servers='localhost:9092')
    mongo_client = MongoClient("mongodb://localhost:27017/")

    db = mongo_client["ftde01"]
    collection = db["project4-collection"]
    print("Starting the consumer")
    
    for msg in consumer:
        
        # Parsing pesan Kafka
        data = json.loads(msg.value)
        
        data_merge = pd.DataFrame([data]).merge(df, how='inner', on=['nameOrig','nameDest'])
        
        prediction = runModel(data_merge.to_dict('index')[0], './modelling')  # Ubah '.' dengan path packages jika diperlukan
        
        # Menambahkan hasil prediksi ke data sebelum disimpan
        data_merge['prediction'] = prediction
        
        collection.insert_one(data_merge.to_dict('index')[0])
        
        print(f"Records = {data_merge.to_dict('index')[0]}")

Starting the consumer
Records = {'step': 1, 'type': 'PAYMENT', 'amount': 9839.64, 'nameOrig': 'C1231006815', 'newbalanceOrig': 160296.36, 'nameDest': 'M1979787155', 'newbalanceDest': 0.0, 'oldbalanceOrg': 170136.0, 'oldbalanceDest': 0.0, 'prediction': 'White List'}
Records = {'step': 1, 'type': 'PAYMENT', 'amount': 1864.28, 'nameOrig': 'C1666544295', 'newbalanceOrig': 19384.72, 'nameDest': 'M2044282225', 'newbalanceDest': 0.0, 'oldbalanceOrg': 21249.0, 'oldbalanceDest': 0.0, 'prediction': 'White List'}
Records = {'step': 1, 'type': 'TRANSFER', 'amount': 181.0, 'nameOrig': 'C1305486145', 'newbalanceOrig': 0.0, 'nameDest': 'C553264065', 'newbalanceDest': 0.0, 'oldbalanceOrg': 181.0, 'oldbalanceDest': 0.0, 'prediction': 'White List'}
Records = {'step': 1, 'type': 'CASH_OUT', 'amount': 181.0, 'nameOrig': 'C840083671', 'newbalanceOrig': 0.0, 'nameDest': 'C38997010', 'newbalanceDest': 0.0, 'oldbalanceOrg': 181.0, 'oldbalanceDest': 21182.0, 'prediction': 'White List'}
Records = {'step': 1, 't

KeyboardInterrupt: 

In [21]:
data_from_mongo = list(collection.find())
    
# Membaca data ke dalam DataFrame
df = pd.DataFrame(data_from_mongo)

# Menghapus kolom '_id'
df = df.drop(columns=['_id'])

# Menampilkan DataFrame tanpa kolom '_id'
df.head()

,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest,oldbalanceOrg,oldbalanceDest,prediction
0,1,PAYMENT,9839.64,C1231006815,160296.36,M1979787155,0.0,170136.0,0.0,White List
1,1,PAYMENT,1864.28,C1666544295,19384.72,M2044282225,0.0,21249.0,0.0,White List
2,1,TRANSFER,181.00,C1305486145,0.00,C553264065,0.0,181.0,0.0,White List
3,1,CASH_OUT,181.00,C840083671,0.00,C38997010,0.0,181.0,21182.0,White List
4,1,PAYMENT,11668.14,C2048537720,29885.86,M1230701703,0.0,41554.0,0.0,White List
